In [103]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import googlemaps

In [104]:
routes_csv = pd.read_csv('../data/routesDist.csv')

In [105]:
usSrcRoutes = routes_csv[routes_csv['SRCCTRY'] == 'United States']
print(len(usSrcRoutes))

usDestinRoutes = routes_csv[routes_csv['DESTINCTRY'] == 'United States']
print(len(usDestinRoutes))

13021
13016


In [106]:
mergeCols = ['AIRLINE_ID', 'SRC_AIRPT_ID', 'DESTIN_AIRPT_ID']

usRoutes = usSrcRoutes.merge(usDestinRoutes, how='outer', on=mergeCols)
len(usRoutes)

15519

In [108]:
usRoutesTest = pd.concat([usSrcRoutes,usDestinRoutes])
usRoutesTest = usRoutesTest.drop_duplicates()
len(usRoutesTest)

15519

In [109]:
#re-using variable above that was used to show concat and outer join provide similar data
usRoutes = usRoutesTest[['SRC_AIRPT_ID', 'SRCNAME', 'DESTIN_AIRPT_ID', 'DESTINNAME']]

In [110]:
def distCalc(x, y):
    file = open("../dataEngineering/gMapsAPIKey.txt")
    gMapsKey = file.read()
    file.close()

    gmaps = googlemaps.Client(key=gMapsKey)

    distance = gmaps.distance_matrix(x, y)['rows'][0]['elements'][0]

    try:
        meters = distance['distance']['value']
    except:
        meters = r'\N'

    try:
        seconds = distance['duration']['value']
    except:
        seconds = r'\N'

    return(meters,seconds)

In [111]:
usRoutes = usRoutes.reset_index()
usRoutes = usRoutes.rename(columns={'index': 'og_index'})

In [117]:
permM = []
permS = []

In [340]:
m = []
s = []

for i in range(7100,7200):
    mVal, sVal = distCalc(usRoutes['SRCNAME'][i], usRoutes['DESTINNAME'][i])
    m.append(mVal)
    s.append(sVal)
    time.sleep(1)
    

In [338]:
permM.append(m)
permS.append(s)

In [339]:
permSeriesM = pd.DataFrame(permM)
permSeriesS = pd.DataFrame(permS)

permSeriesM.to_csv('../data/meters.csv')
permSeriesS.to_csv('../data/seconds.csv')

In [336]:
usRoutes.to_csv('../data/usRoutes.csv')